# Simple Ising Model Example

This notebook demonstrates:
1. Creating an Ising model (from SpinSystems submodule)
2. Setting up a Metropolis algorithm with Boltzmann weights
3. Configuring measurements with the new measurement framework
4. Running equilibrium Monte Carlo simulation
5. Analyzing results

In [ ]:
using Random
using StatsBase
using MonteCarloX
using MonteCarloX.SpinSystems

## Setup System and Algorithm

In [ ]:
# Setup
rng = MersenneTwister(42)
L = 8
β = 0.4  # Inverse temperature

println("System configuration:")
println("  Lattice size: $(L)×$(L)")
println("  Inverse temperature β: $β")
println("  Total spins: $(L*L)")

In [ ]:
# Create Ising system
sys = Ising([L, L], J=1, periodic=true)
init!(sys, :random, rng=rng)

# Create Metropolis algorithm with Boltzmann weight
alg = Metropolis(rng, β=β)

## Configure Measurements

In [ ]:
# Setup measurements
measurements = Measurements([
    :energy => energy => Float64[],
    :magnetization => magnetization => Float64[]
], interval=10)

## Run Simulation

In [ ]:
# Thermalization
println("Thermalizing...")
N = L * L
for i in 1:N*1000
    spin_flip!(sys, alg)
end
println("  Thermalization complete: $(alg.steps) steps")

# Reset statistics after thermalization
reset_statistics!(alg)

In [ ]:
# Production run
println("Production run...")
for i in 1:N*10000
    spin_flip!(sys, alg)
    measure!(measurements, sys, i)
end

## Analyze Results

In [ ]:
energies = measurements[:energy].data
mags = measurements[:magnetization].data

avg_E = mean(energies) / N
avg_M = mean(mags) / N

println("\nResults:")
println("  Total steps: $(alg.steps)")
println("  Acceptance rate: $(round(acceptance_rate(alg), digits=4))")
println("\nObservables (per spin):")
println("  Average energy: $(round(avg_E, digits=4))")
println("  Average magnetization: $(round(avg_M, digits=4))")